# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Откроем датафрейм,ознакомимся с данными
data = pd.read_csv('/datasets/users_behavior.csv')
print(data.head())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# смотрим колонку согласно задания, в нем 0 и 1 
data['is_ultra'].unique()

array([0, 1])

In [5]:
# смотрим дубликаты
data.duplicated().sum()

0

## Разбейте данные на выборки

In [6]:
data_t, data_test = train_test_split(data, test_size=0.2, random_state=12345)
data_train, data_valid = train_test_split(data_t, test_size=0.25, random_state=12345)

In [7]:
print(data_train.info())
print(data_test.info())
data_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2656 to 510
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1415 to 1196
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2699 to 1806
Data columns (total 5 columns):
 #   Column    Non

In [8]:
# разделим датафрейм, используя функцию train_test_split на три выборки

## Исследуйте модели

In [9]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

In [10]:
# дерево решений
best_result = 0
depth = 0
depths = []
results = []

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    depths.append(depth)
    results.append(result)
    if result > best_result:
        best_depth = depth
        best_result = result

print(best_result, best_depth)

0.7651632970451011 3


In [11]:
# наибольшая доля правильных ответов наблюдается у дерева с глубиной 3 - 76.5%

In [12]:
# случайный лес
best_model = None
best_result = 0
for est in range(10, 90, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print(best_result, best_est)

0.7993779160186625 80


In [13]:
# логистическая регрессия
model_lr = LogisticRegression()
model_lr.fit(features_train, target_train)
predictions_valid = model_lr.predict(features_valid)
accuracy = model_lr.score(features_valid, target_valid)
accuracy

0.7262830482115086

Доля правильных ответов в модели логистической регрессии - 72.6%

## Проверьте модель на тестовой выборке

In [14]:
# model = DecisionTreeClassifier(max_depth=9, random_state=1)
# model.fit(features_train, target_train)
# predictions_test = model.predict(features_test)
# accuracy_score(target_test, predictions_test)

In [15]:
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

In [16]:
features_train_t = data_t.drop(['is_ultra'], axis=1)
target_train_t = data_t['is_ultra']

model_forest = RandomForestClassifier(random_state=12345, n_estimators=80)
model_forest.fit(features_train_t, target_train_t)

predictions_test_forest = model_forest.predict(features_test)
accuracy_test_forest = accuracy_score(target_test, predictions_test_forest)
accuracy_test_forest

0.7713841368584758

На тестовой выборке выбранная модель показывает точность близкую к валидационной, но не больше. Это означает отсутствие проблем с переобучение модели.

## Проверьте модели на адекватность

In [17]:
data['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [18]:
data_test['is_ultra'].value_counts()

0    447
1    196
Name: is_ultra, dtype: int64

In [19]:
data['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Доля большего класса тестовой выборки равна ~0.69 (Смарт); полученная нами модель имеет accuracy ~0.79. Таким образом, мы можем считать модель адекватной для использования

Вывод

Наиболее адекватные предсказания делает Random Forest, это видно по точности предсказания на валидационной и на тестовой выборке

В исследовании участвовали три различные алгоритма классификации:случайный лес, дерево решений, логистическая регрессия.
Для проведения обучения, проверки моделей и тестирования лучшей модели исходный датафрейм был разделён на три выборки: обучающую (60%), валидационную (20%) и тестовую (20%).
При проверке модели на адекватность сделан вывод, что она показывает намного лучший результат, чем модель с примитивным прогнозом.
По итогам исследования можно сделать вывод о том, что для получения рекомендации тарифных планов наиболее подходящей является модель - Random Forest,  с количеством деревьев 80, где доля правильных ответов в тестовой выборке составила 77.1%